In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import LSTM, GRU, Dense, BatchNormalization
from tensorflow.keras import backend
from tqdm.notebook import tqdm
import plotly.express as px
import pandas as pd

##### Data preparation

In [ ]:
x = np.load('x.npy')
y = np.load('y.npy')
y_forced = np.zeros(shape=y.shape)
for i in tqdm(range(y_forced.shape[0])):
    y_forced[i] = np.roll(y[i], 1)
    y_forced[i][0] = 0
x = np.expand_dims(x, axis=2)
y = np.expand_dims(y, axis=2)
y_forced = np.expand_dims(y_forced, axis=2)
print(x.shape, y.shape, y_forced.shape)

##### Model definition

In [ ]:
backend.clear_session()
# Dimensions
encoder_timesteps = 27
decoder_timesteps = 6
latent_dim = 16
n_features = 1

# Encoder
encoder_in = Input(shape=(encoder_timesteps, n_features), name='encoder_in')
gru_encoder = GRU(units=latent_dim, return_state=True, name='encoder_lstm')
_, encoder_state = gru_encoder(encoder_in)

# Decoder
decoder_in = Input(shape=(decoder_timesteps, n_features), name='decoder_in')
gru_decoder = GRU(latent_dim, return_sequences=True, return_state=True, name='decoder_gru')
decoder_out_training, _ = gru_decoder(decoder_in, initial_state=encoder_state)

# Decoder in prediction mode
decoder_state_in = Input(shape=(latent_dim,), name='decoder_state_in')
decoder_out_prediction, decoder_state_out = gru_decoder(decoder_in, initial_state=decoder_state_in)

# Dense output
decoder_dense = Dense(1, activation='sigmoid')
decoder_out_training = decoder_dense(decoder_out_training)
decoder_out_prediction = decoder_dense(decoder_out_prediction)

# Training model (allowing teacher forcing)
training_model = Model([encoder_in, decoder_in], decoder_out_training)

# Prediction model
encoder_model = Model(encoder_in, encoder_state)
decoder_model = Model([decoder_in, decoder_state_in], [decoder_out_prediction, decoder_state_out])

# Summary
print(training_model.summary())
print(encoder_model.summary())
print(decoder_model.summary())

##### Training

In [ ]:
epochs = 10
training_model.compile(optimizer='rmsprop', loss='mse', metrics=['mse', 'mae'])
training_model.fit([x, y_forced], y,
          epochs=epochs,
          batch_size=1024,
          validation_split=0.3
         )

In [ ]:
y_pred = training_model.predict([x[:1000], y_forced[:1000]])
data = pd.DataFrame({'y': y[:1000].flatten(), 'y_pred': y_pred.flatten()}).reset_index().melt(id_vars='index').sort_values('index')
px.line(data, x='index', y='value', color='variable')

##### Validation